<a href="https://colab.research.google.com/github/harsh-hy/DeepLearning/blob/main/vanilla_denoising__autoEncoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.losses import binary_crossentropy

In [ ]:
# Load the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize and flatten the images
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train,(len(x_train),28,28,1))
x_test = np.reshape(x_test,(len(x_test),28,28,1))

print(x_train.shape)
print(x_test.shape)

In [ ]:
#vanilla
input_img=Input(shape=(28,28,1))


x=Flatten()(input_img)
encoded = Dense(64, activation='relu')(x)
decoded = Dense(784, activation='sigmoid')(encoded)
decoded=Reshape((28,28,1))(decoded)

# Autoencoder model
vanilla_autoencoder = Model(input_img, decoded)

# Compile the model
vanilla_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
print("TRAINING ....")
vanilla_autoencoder.fit(x_train, x_train,
                epochs=6,
                batch_size=256,
                validation_data=(x_test, x_test),
                verbose=1)

In [ ]:
decoded_imgs_vanilla=vanilla_autoencoder.predict(x_test[:5])

In [ ]:
# Denoising Autoencoder

# Add noise to the images
noise_factor = 0.3
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the values to be between 0 and 1
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

x_train_noisy = np.reshape(x_train_noisy,(len(x_train_noisy),28,28,1))
x_test_noisy = np.reshape(x_test_noisy,(len(x_test_noisy),28,28,1))
x_train = np.reshape(x_train,(len(x_train),28,28,1))
x_test = np.reshape(x_test,(len(x_test),28,28,1))


# Define the denoising autoencoder model
input_img_denoising = Input(shape=(28, 28, 1))


In [ ]:
x=Flatten()(input_img_denoising)
encoded_denoising = Dense(64, activation='relu')(x)
decoded_denoising = Dense(784, activation='sigmoid')(encoded_denoising)
decoded_denoising=Reshape((28,28,1))(decoded_denoising)

denoising_autoencoder = Model(input_img_denoising, decoded_denoising)
denoising_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
print("TRAINING ...")
denoising_autoencoder.fit(x_train_noisy, x_train,
                epochs=6,
                batch_size=256,
                validation_data=(x_test_noisy, x_test),
                verbose=1)

In [ ]:
denoising_imgs_denoising=denoising_autoencoder.predict(x_test_noisy[:5])

In [ ]:
#VISUAL COMPARISON
n=5
plt.figure(figsize=(20,4))
for i in range(n):
  #original
  ax=plt.subplot(3,n,i+1)
  plt.imshow(x_test[i].reshape(28,28))
  plt.title("ORIGINAL")
  plt.axis("off")

  #vanilla
  ax=plt.subplot(3,n,i+1+n)
  plt.imshow(decoded_imgs_vanilla[i].reshape(28,28))
  plt.title("VANILLA")
  plt.axis("off")

  #denoising
  ax=plt.subplot(3,n,i+1+2*n)
  plt.imshow(denoising_imgs_denoising[i].reshape(28,28))
  plt.title("DENOISING")
  plt.axis("off")